In [ ]:
## OLIST Project - Objective Develop a framework for REF Analysis
## Recommender System

In [ ]:
# import the respective libraries required for this project

import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

## Uploading the data, merging and some setting

In [ ]:

pd.options.display.max_columns = 999

In [ ]:
# uploading data 

customers = pd.read_csv("olist_customers_dataset.csv")
geolocation =pd.read_csv("olist_geolocation_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
order_payments = pd.read_csv("olist_order_payments_dataset.csv")
order = pd.read_csv("olist_orders_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
product_category_name = pd.read_csv("product_category_name_translation.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
sellers  = pd.read_csv("olist_sellers_dataset.csv")


In [ ]:
# Merging Dataframes
df = order.merge(order_items, on='order_id', how='inner')
df = df.merge(order_payments, on='order_id', how='inner', validate='m:m')
df = df.merge(reviews, on='order_id', how='inner')
df = df.merge(products, on='product_id', how='inner')
df = df.merge(customers, on='customer_id', how='inner')
df = df.merge(sellers, on='seller_id', how='inner')

## EDA

In [ ]:
df.head()

In [ ]:
print(df.shape)
df.info()

In [ ]:
df.isna().sum()

In [ ]:
# order_purchase_timestamp
# order_delivered_customer_date

df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'])
df['order_delivered_customer_date'] = pd.to_datetime(df['order_delivered_customer_date'])

In [ ]:
# extracting day, week ,Month like features from these columns

df['order_purchase_day'] = df['order_purchase_timestamp'].dt.day
df['order_purchase_month'] = df['order_purchase_timestamp'].dt.month
df['order_purchase_week'] = df['order_purchase_timestamp'].dt.isocalendar().week
df['order_purchase_weekday'] = df['order_purchase_timestamp'].dt.weekday+1 # i here represents Monday
df['order_purchase_year'] = df['order_purchase_timestamp'].dt.year

# calculating the delivery time - 

df['Delivery_time'] = (df['order_delivered_customer_date']-df['order_purchase_timestamp']).dt.days # in terms of Number of Days

In [ ]:
df.head()

In [ ]:
# what can we do here

monthly_sales = df.groupby(df['order_purchase_timestamp'].dt.to_period('M')).agg({'price' : 'sum'})

#plotting the monthly sales
plt.figure(figsize=(9,8))

plt.plot(monthly_sales.index.astype(str),monthly_sales['price'], 
         marker='o', linestyle='-', color='blue', linewidth=2)

# Adding titles and labels
plt.title('Monthly Sales Trend', fontsize=16, fontweight='bold')
plt.xlabel('Month', fontsize=14)
plt.ylabel('Total Sales', fontsize=14)

# Customize the ticks
plt.xticks(rotation=45, fontsize=12)
plt.yticks(fontsize=12)

plt.tight_layout()
plt.show()

In [ ]:
# Total Sales by product category

sales_by_area = df.groupby('customer_state')['order_id'].count()

plt.figure(figsize=(15,8))

sales_by_area.plot(kind='bar', color = 'blue',edgecolor='black')

# Adding title
plt.title('Sales Distribution by State', fontsize=16, fontweight='bold')

# Hide the y-label for better aesthetics
plt.ylabel('')
plt.xticks(rotation=45, fontsize=12)
# Add gridlines
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()
plt.tight_layout()
plt.show()

In [ ]:
# Set the figure size
plt.figure(figsize=(8, 8))

sales_by_payment = df.groupby('payment_type')['price'].sum()
sales_by_payment.plot(kind='pie', autopct='%1.1f%%', startangle=90)

# Adding title
plt.title('Sales Distribution by Payment Type', fontsize=16, fontweight='bold')

# Hide the y-label for better aesthetics
plt.ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# pllotiing a delivery time 
plt.figure(figsize=(8, 6))
sns.violinplot(x=df['Delivery_time'], color='lightblue')
plt.title('Violin Plot of Delivery Time')
plt.xlabel('Delivery Time')
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))

# Plot the histogram
sns.histplot(df['Delivery_time'], bins=50, kde=True, color='blue', edgecolor='black')

# Add title and labels
plt.title('Combined Histogram and Density Plot of Delivery Time')
plt.xlabel('Delivery Time')
plt.ylabel('Frequency/Density')

# Show the plot
plt.show()

In [ ]:
# plotting the delivery time for each month - column -> order_purchase_month

month_delivery_time_avg = df.groupby('order_purchase_month')['Delivery_time'].mean().reset_index()

# Plot
plt.figure(figsize=(12, 6))
sns.barplot(x='order_purchase_month', y='Delivery_time', data=month_delivery_time_avg)
plt.title('Average Delivery Time by Month', fontsize=16, fontweight='bold')
plt.xlabel('Month-Year', fontsize=14)
plt.ylabel('Average Delivery Time (Days)', fontsize=14)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


## REF Analysis

In [ ]:
# for checking data type
df.info()

In [ ]:
# to check what is the most recent date on which an order was placed - the refrence date should be ahead of this date

df['order_purchase_timestamp'].max()

In [ ]:
# the last date for purchase in the data set is 2018-03-08 - this was done to create a benchmark for calculating the recency factor

# thus defining the refrence data

reference_date = datetime(2018, 12, 31) # this is the date from which we will calculate the recency analysis -  let assume that this was the day that analysis was done

recency_df = df.groupby('customer_id')['order_purchase_timestamp'].max().reset_index()

recency_df['Recency'] = (reference_date-recency_df['order_purchase_timestamp']).dt.days

recency_df.head()


In [ ]:
# calculating the frequncy - the number of orders purchased by the customer

frequency_df = df.groupby('customer_id')['order_id'].nunique().reset_index()
frequency_df.columns = ['customer_id', 'Frequency']
frequency_df.head()

In [ ]:
# caculaating the monetary value for each customer

value_df = df.groupby("customer_id")['price'].sum().reset_index()
value_df.columns = ['customer_id', 'Monetary Value']
value_df.head()

In [ ]:
#merge the values

ref_metrics = recency_df.merge(frequency_df, on ="customer_id")
ref_metrics  = ref_metrics.merge(value_df, on="customer_id")
ref_metrics.head()

### K Mean Clustering

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Scale the RFM values
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(ref_metrics[['Recency', 'Frequency', 'Monetary Value']])

# Apply K-Means
kmeans = KMeans(n_clusters=4, random_state=42)
ref_metrics['Cluster'] = kmeans.fit_predict(rfm_scaled)

# View cluster distributions
cluster_counts = ref_metrics['Cluster'].value_counts()


In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# 3D scatter plot
scatter = ax.scatter(
    ref_metrics['Recency'], 
    ref_metrics['Frequency'], 
    ref_metrics['Monetary Value'], 
    c=ref_metrics['Cluster'], 
    cmap='viridis', 
    s=100, alpha=0.7
)

# Labels and title
ax.set_title('RFM Clusters - 3D Visualization')
ax.set_xlabel('Recency')
ax.set_ylabel('Frequency')
ax.set_zlabel('Monetary')
plt.colorbar(scatter, label='Cluster')
plt.show()


In [ ]:
# plotting the clusters using Plotly
fig = px.scatter_3d(
    ref_metrics, 
    x='Recency', 
    y='Frequency', 
    z='Monetary Value', 
    color='Cluster', 
    title='RFM Clusters - 3D Visualization',
    size_max=10,
    opacity=0.7
)
fig.show()
